## Load Data

In [ ]:
import pandas as pd
import numpy as np
import pickle
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from scipy import stats

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [79]:
def load_wesad_subject(subject_path):
    
    with open(subject_path, 'rb') as file:
        data = pickle.load(file, encoding='latin1')

    
    chest_data = data['signal']['chest']
    
    
    df = pd.DataFrame()
    df['ACC_x'] = chest_data['ACC'][:, 0]
    df['ACC_y'] = chest_data['ACC'][:, 1]
    df['ACC_z'] = chest_data['ACC'][:, 2]
    df['ECG']   = chest_data['ECG'].flatten()
    df['EMG']   = chest_data['EMG'].flatten()
    df['EDA']   = chest_data['EDA'].flatten()
    df['Temp']  = chest_data['Temp'].flatten()
    df['Resp']  = chest_data['Resp'].flatten()
    
    
    df['label'] = data['label']
    df['subject'] = data['subject']
    
    return df

In [80]:
def create_windows(df, window_seconds=2, stride_seconds=0.25, sampling_rate=700):
    
    window_steps = int(window_seconds * sampling_rate)
    stride_steps = int(stride_seconds * sampling_rate)
    
    feature_cols = ['ACC_x', 'ACC_y', 'ACC_z', 'ECG', 'EMG', 'EDA', 'Temp', 'Resp']
    data = df[feature_cols].values
    labels = df['label'].values
    
    X_windows = []
    y_windows = []
    
    
    for i in range(0, len(df) - window_steps, stride_steps):
        window_data = data[i : i + window_steps]
        window_labels = labels[i : i + window_steps]
        
        
        mode_label = stats.mode(window_labels, keepdims=True)[0][0]
        
        
        X_windows.append(window_data.transpose()) 
        y_windows.append(mode_label)
        
    return np.array(X_windows), np.array(y_windows)

In [85]:

base_path = r"H:\Research\archive (1)\WESAD"

subject_id = "S2"
file_path = os.path.join(base_path, subject_id, f"{subject_id}.pkl")

if os.path.exists(file_path):
    print(f"Loading {subject_id}...")
    df = load_wesad_subject(file_path)

    df = df[df['label'].isin([1, 2])].copy()
    
    label_map = {1: 0, 2: 1}
    df['label'] = df['label'].map(label_map)
    
    print("Creating windows...")
    X, y = create_windows(df, window_seconds=2, stride_seconds=0.25)
    print(f"Window shape: {X.shape}") # Should be (N, 8, 1400)


    N, C, T = X.shape
    
   
    X_flat = X.transpose(0, 2, 1).reshape(-1, C)
    scaler = StandardScaler()
    X_scaled_flat = scaler.fit_transform(X_flat)
    X_norm = X_scaled_flat.reshape(N, T, C).transpose(0, 2, 1)
    
    print(f"Data Normalized. Mean: {X_norm.mean():.2f}, Std: {X_norm.std():.2f}")

else:
    print(f"Error: File not found at {file_path}")

Loading S2...
Creating windows...
Creating windows...
Window shape: (7028, 8, 1400)
Window shape: (7028, 8, 1400)
Data Normalized. Mean: -0.00, Std: 1.00
Data Normalized. Mean: -0.00, Std: 1.00


In [86]:
import numpy as np

print("Old unique labels:", np.unique(y_s2))

# 1. Identify indices where label is either 1 (Baseline) or 2 (Stress)
# We ignore 0 (Transient), 3 (Amusement), 4 (Meditation) for this binary task
valid_indices = np.where((y_s2 == 1) | (y_s2 == 2))[0]

# 2. Keep only those valid windows
X_final = X_s2[valid_indices]
y_final = y_s2[valid_indices]

# 3. Remap the labels to be 0 and 1
# Original 1 becomes 0
# Original 2 becomes 1
y_final = np.where(y_final == 1, 0, 1)

print("New unique labels (should be [0 1]):", np.unique(y_final))
print(f"Final dataset shape: {X_final.shape}")

Old unique labels: [0 1 2 3 4 6 7]
New unique labels (should be [0 1]): [0 1]
Final dataset shape: (7036, 8, 1400)
New unique labels (should be [0 1]): [0 1]
Final dataset shape: (7036, 8, 1400)


In [87]:
# Convert to Tensors
tensor_x = torch.Tensor(X_norm)
tensor_y = torch.Tensor(y).long()

# Time-Based Split (First 80% Train, Last 20% Test)
split_idx = int(0.8 * len(tensor_x))

X_train = tensor_x[:split_idx]
y_train = tensor_y[:split_idx]
X_test = tensor_x[split_idx:]
y_test = tensor_y[split_idx:]

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=False)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=batch_size, shuffle=False)

print(f"Train samples: {len(X_train)} | Test samples: {len(X_test)}")

Train samples: 5622 | Test samples: 1406


In [88]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

# 1. Update Tensors with the FIXED data
tensor_x = torch.Tensor(X_final) 
tensor_y = torch.Tensor(y_final).long() # .long() is required for labels

# 2. Split again
X_train, X_test, y_train, y_test = train_test_split(tensor_x, tensor_y, test_size=0.2, random_state=42)

# 3. Re-create DataLoaders
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("DataLoaders updated. You can now run the training loop.")

DataLoaders updated. You can now run the training loop.


In [89]:
class LightweightModel(nn.Module):
    def __init__(self, input_channels=8, num_classes=2, window_size=1400):
        super(LightweightModel, self).__init__()
        
        # Encoder (Client Side)
        self.encoder = nn.Sequential(
            nn.Conv1d(input_channels, 16, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(16, 32, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )
        
        # Classifier (Server Side)
        final_dim = window_size // 4 
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * final_dim, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.classifier(x)
        return x

model = LightweightModel().to(device)
print(model)

LightweightModel(
  (encoder): Sequential(
    (0): Conv1d(8, 16, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(16, 32, kernel_size=(5,), stride=(1,), padding=(2,))
    (4): ReLU()
    (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=11200, out_features=64, bias=True)
    (2): ReLU()
    (3): Linear(in_features=64, out_features=2, bias=True)
  )
)


In [90]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
epochs = 10

print("Starting Training...")

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print(f"Epoch {epoch+1} | Loss: {running_loss/len(train_loader):.4f} | Acc: {100*correct/total:.2f}%")

print("Training Complete.")

Starting Training...
Epoch 1 | Loss: 0.7344 | Acc: 64.39%
Epoch 1 | Loss: 0.7344 | Acc: 64.39%
Epoch 2 | Loss: 0.6564 | Acc: 64.64%
Epoch 2 | Loss: 0.6564 | Acc: 64.64%
Epoch 3 | Loss: 0.6513 | Acc: 64.64%
Epoch 3 | Loss: 0.6513 | Acc: 64.64%
Epoch 4 | Loss: 0.6501 | Acc: 64.64%
Epoch 4 | Loss: 0.6501 | Acc: 64.64%
Epoch 5 | Loss: 0.6498 | Acc: 64.64%
Epoch 5 | Loss: 0.6498 | Acc: 64.64%
Epoch 6 | Loss: 0.6497 | Acc: 64.64%
Epoch 6 | Loss: 0.6497 | Acc: 64.64%
Epoch 7 | Loss: 0.6497 | Acc: 64.64%
Epoch 7 | Loss: 0.6497 | Acc: 64.64%
Epoch 8 | Loss: 0.6498 | Acc: 64.64%
Epoch 8 | Loss: 0.6498 | Acc: 64.64%
Epoch 9 | Loss: 0.6497 | Acc: 64.64%
Epoch 9 | Loss: 0.6497 | Acc: 64.64%
Epoch 10 | Loss: 0.6497 | Acc: 64.64%
Training Complete.
Epoch 10 | Loss: 0.6497 | Acc: 64.64%
Training Complete.
